In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Co potrzebujemy do opisu pozycji na ETF?

Spróbujmy tak:
1. Czas otwarcia pozycji
2. Ticker instrumentu

Dodatkowe parametry:
1. Spread walutowy (ok. `0.002`), SPREAD
2. Marża za przewalutowanie (`0.5%`), MARGIN

Przykład: otwieramy pozycję na `4GLD.DE`

In [2]:
def get_exchange_rate(currency: str, time: pd.Timestamp, type: str | None, spread: float | None, margin: float | None):
    
    # Assertions
    assert type in ('bid', 'ask', None), ''
    if type in ('bid', 'ask'): assert spread is not None and margin is not None, ''
    
    # Code
    if currency not in ('PLN'): # jeżeli kupujemy instrument w obcej walucie
    
        currency_ticker = f'{currency}PLN=X'
        currency_ticker = yf.Ticker(currency_ticker)
        
        currency_history = currency_ticker.history(
            start=time.floor('h'), 
            end=time.ceil('h'),
            interval='1m'
        )
        currency_history = currency_history[currency_history.index == time].iloc[0]
        
        if type == 'ask':
            exchange_rate = round((currency_history['High'] + spread / 2) * (1 + margin), 4)
        elif type == 'bid':
            exchange_rate = round((currency_history['Low'] - spread / 2) * (1 - margin), 4)
        else:
            exchange_rate = 0.5*(currency_history['High'] + currency_history['Low'])
        
    else:
        exchange_rate = None
        
    return exchange_rate


get_exchange_rate(currency='EUR', time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), type='ask', spread=0.002, margin=0.005)

np.float64(4.2321)

In [16]:
def get_ticker_price(ticker: str, time: pd.Timestamp, type: str, spread: float):
    
    # Assertions
    assert type in ('bid', 'ask', None), ''
    if type in ('bid', 'ask'): assert spread is not None, ''
    
    # Code
    
    ticker = yf.Ticker(ticker)
    
    ticker_history = ticker.history(
        start=time.floor('h'), 
        end=time.ceil('h'),
        interval='1m'
    )

    ticker_history = ticker_history[ticker_history.index == time].iloc[0]
    if type == 'ask':
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low']) * (1 + spread)
    elif type == 'bid':
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low']) * (1 - spread)
    else:
        ticker_price = 0.5*(ticker_history['High'] + ticker_history['Low'])

    return ticker_price

get_ticker_price(ticker='4GLD.DE', time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), type='ask', spread=0.001)

np.float64(125.87074393081663)

In [ ]:
# Input
position_opening_time = '2026-02-02T09:22'
volume = 0.3492
ticker = '4GLD.DE'

MARGIN = 0.005

# Code
position_opening_time = pd.Timestamp(position_opening_time, tz='Europe/Warsaw')

ticker = yf.Ticker(ticker)
info = ticker.info

currency = info['currency']
timezone = info['exchangeTimezoneName']
position_opening_time = position_opening_time.tz_convert(timezone)

# Parametry przy otwarciu pozycji
position_opening_exchange_rate = get_exchange_rate(
    currency=currency, 
    time=position_opening_time, 
    type='ask', 
    spread=0.002, # hardcoded
    margin=MARGIN
)

position_opening_ticker_price = get_ticker_price(
    ticker='4GLD.DE', 
    time=pd.Timestamp('2026-02-02T09:22', tz='Europe/Warsaw'), 
    type='ask', 
    spread=0.001 # hardcoded
)

position_opening_value = position_opening_ticker_price * position_opening_exchange_rate * volume
position_opening_value

np.float64(186.01799332605148)

In [ ]:
get_exchange_rate(
    currency='EUR', 
    time=pd.Timestamp.now(tz='Europe/Warsaw'), 
    type='ask', 
    spread=SPREAD, 
    margin=MARGIN
)